In [1]:
import RPi.GPIO as GPIO
import time

# 사용자 정의 핀 번호 설정
TRIG_PIN = 23
ECHO_PIN = 24

In [2]:
# GPIO 핀을 초기화하는 함수
def setup_gpio(trig_pin, echo_pin):
    # BCM 모드로 설정(GPIO 핀번호 기반 모드)
    GPIO.setmode(GPIO.BCM)
    # 트리거 핀을 출력 모드로 설정
    GPIO.setup(trig_pin, GPIO.OUT)
    # 에코 핀을 입력 모드로 설정
    GPIO.setup(echo_pin, GPIO.IN)

In [3]:
# 초음파 트리거 펄스를 보내는 함수
def send_trigger_pulse(trig_pin):
    # 트리거 초기화
    GPIO.output(trig_pin, False)
    time.sleep(0.1)  # 안정화 지연
    # 트리거를 활성화하여 매우 짧은 펄스를 보냄
    GPIO.output(trig_pin, True)
    time.sleep(0.00001)
    # 트리거를 비활성화
    GPIO.output(trig_pin, False)

In [4]:
# 에코 신호를 대기하는 함수
def wait_for_echo(echo_pin, value, timeout):
    count = timeout
    # 지정된 값이 입력될 때까지 반복, 타임아웃 방지
    while GPIO.input(echo_pin) != value and count > 0:
        count -= 1

In [5]:
# 트리거 및 에코 핀의 펄스 지속 시간을 측정하는 함수
def get_pulse_time(trig_pin, echo_pin):
    send_trigger_pulse(trig_pin)
    wait_for_echo(echo_pin, 1, 10000)  # 에코 핀이 HIGH가 되는 것을 대기
    start = time.time()  # 시작 시간 기록
    wait_for_echo(echo_pin, 0, 10000)  # 에코 핀이 LOW가 되는 것을 대기
    finish = time.time()  # 종료 시간 기록
    pulse_len = finish - start  # 펄스 길이 계산
    return pulse_len

In [6]:
# 초음파 센서를 사용하여 거리를 계산하는 함수
def calculate_distance(trig_pin, echo_pin):
    pulse_len = get_pulse_time(trig_pin, echo_pin)
    distance_cm = pulse_len * 34300 / 2.0  # 공기 중 음속을 사용하여 센티미터로 변환
    return distance_cm / 100.0  # 미터 단위로 변환

In [7]:
# 주어진 핀을 사용하여 거리를 계속 측정하는 함수
def measure_distance(trig_pin, echo_pin):
    try:
        setup_gpio(trig_pin, echo_pin)

        while True:
            distance = calculate_distance(trig_pin, echo_pin)
            print(f"Distance: {distance:.2f} meters")
            time.sleep(1)  # 1초 대기 후 다시 측정

    except KeyboardInterrupt:
        # 사용자가 중단했을 때 메시지 출력
        print("\nMeasurement stopped by user")

    finally:
        # GPIO 핀 정리
        GPIO.cleanup()

# 거리 측정 함수 호출
measure_distance(TRIG_PIN, ECHO_PIN)

Distance: 0.17 meters
Distance: 0.20 meters
Distance: 0.08 meters
Distance: 0.08 meters
Distance: 0.09 meters
Distance: 0.17 meters
Distance: 0.17 meters

Measurement stopped by user
